# REDES NEURAIS

O objetivo desse *notebook* é apresentar os fundamentos matemáticos que baseiam as redes neurais. Para isso, nós iremos implementar uma rede neural e usar o Método de Newton para encontrar uma aproximação numérica para os parâmetros que melhor ajustam uma rede neural a um certo conjunto de dados.


Uma *rede neural* é uma função $f : X \to Y$ que geralmente é *ajustada* (ou *treinada*) para modelar a relação entrada-saída de um conjunto de dados. Nesse caso, o domínio da rede neural (o conjunto $X$) deve conter os vetores-entrada, o contradomínio (o conjunto $Y$) deve conter os vetores-saída desse conjunto de dados, e os valores da rede neural ajustada aplicada aos vetores-entrada devem estar o mais próximo possível dos respectivos vetores-saída.


Existem diversos tipos de redes neurais. O mais simples é chamado de *perceptron* (ou, em geral, *multi-layer perceptron*). Nesse caso mais simples, a função $f$ é uma composição de várias funções,
$$
    f = f_n \circ f_{n-1} \circ \dotsb \circ f_2 \circ f_1,
$$
onde cada uma das funções $f_1, f_2, \dotsc, f_n$ é chamada de *camada de $f$*. Além disso, cada função $f_i$ também pode ser decomposta como uma composição: de uma multiplicação por uma matriz (chamada de *pesos*), com a soma de um vetor (chamado de *viéses*), com a aplicação de uma *função de ativação*. Nas próximas seções, nós veremos como isso funciona em mais detalhes.


## Rede neural

Nessa seção, vamos implementar uma rede neural. Para isso, precisaremos importar algumas bibliotecas, escolher as camadas da rede neural e escolher a função de ativação que será usada. A partir dessas informações, definiremos os parâmetros da rede e a função *rede neural*.


### Pacotes e bibliotecas

Vamos começar importando um pacote e uma biblioteca de Python: numpy e sympy. O primeiro será usada para cálculos numéricos e a segunda será usada para realizar manipulações simbólicas.


In [ ]:
import numpy as np
import sympy as sp

### Dimensões das camadas

Agora vamos definir as dimensões das camadas da rede neural, ou seja, vamos escolher uma lista de números naturais $(d_0, d_1, \dotsc, d_n)$, onde $d_0$ é a dimensão da camada de entrada, $d_1$ é a dimensão da primeira camada oculta, $\dotsc$ , e $d_n$ é a dimensão da camada de saída. Observe que o caso mais simples possível é aquele em que $d_0 = d_1 = 1$. Nesse caso, temos 1 neurônio na camada de entrada e 1 neurônio na camada de saída.


In [ ]:
# Definir dimensões das camadas:

dimensoes_camadas = [1, 1]

Antes de prosseguirmos, vamos verificar se a lista acima contem pelo menos duas camadas e se a dimensão de cada camada é um número inteiro maior que zero. Se a validação funcionar, vamos exibir uma ilustração da rede neural que será construída.


In [ ]:
# Validação da lista de dimensões

if len(dimensoes_camadas) < 2:
    raise ValueError("A rede neural precisa de pelo menos 2 camadas.")
else:
    for dim in dimensoes_camadas:
        if not isinstance(dim, int):
            raise ValueError(f"A dimensão de uma das camadas não é um número inteiro: {dim}.")
        elif dim < 1:
            raise ValueError(f"A dimensão de uma das camadas não é maior que zero: {dim}.")


# Visualização da rede neural

!pip install graphviz -qq
import graphviz

grafico = graphviz.Digraph(comment='Rede Neural', graph_attr={'rankdir': 'LR'})

nodes_by_layer = []
for k in range(len(dimensoes_camadas)):
    layer_nodes = []
    with grafico.subgraph(name=f'rank_{k}') as s:
        s.attr(rank='same')
        if k == 0:
            layer_nodes = [f'input_{i}' for i in range(dimensoes_camadas[k])]
            node_color = 'lightblue'
        elif k == len(dimensoes_camadas) - 1:
            layer_nodes = [f'output_{i}' for i in range(dimensoes_camadas[k])]
            node_color = 'lightcoral'
        else:
            layer_nodes = [f'hidden_{k}_{i}' for i in range(dimensoes_camadas[k])]
            node_color = 'lightgray'

        nodes_by_layer.append(layer_nodes)

        for node_name in layer_nodes:
             s.node(node_name, shape='circle', label='', width='0.2', height='0.2', color=node_color, style='filled')

for k in range(len(dimensoes_camadas) - 1):
    current_layer_nodes = nodes_by_layer[k]
    next_layer_nodes = nodes_by_layer[k+1]

    for source_node in current_layer_nodes:
        for target_node in next_layer_nodes:
            grafico.edge(source_node, target_node, arrowhead='none', color='gray')

display(grafico)

### Parâmetros

Com as dimensões das camadas da rede neural escolhidas, nós podemos definir os parâmetros da rede neural. Para cada $k \in \{0, 1, \dotsc, n-1\}$, as variáveis $w_{kij}$ representam as entradas da matriz de pesos $w_k = \left( {w_k}_{ij} \right)_{ij}$ e as variáveis $b_{kj}$ representam as coordenadas do vetor de viés $b_k = ({b_k}_1, \dotsc, {b_k}_{d_k})$ da $k$-ésima camada. Por exemplo, no caso mais simples (em que $d_0 = d_1 = 1$), os parâmetros são apenas $w_{000}$ e $b_{00}$.


In [ ]:
# Parâmetros da rede neural

profundidade = len(dimensoes_camadas)-1

## w[k][i][j] é o peso do link j -> i na camada k
w = [[[sp.Symbol(f"w{k}{i}{j}") for j in range(dimensoes_camadas[k])]  # Corrigido: Symbol com S maiúsculo
      for i in range(dimensoes_camadas[k+1])]
      for k in range(profundidade)]

## b[k][j] é o viés somado à coordenada j na camada k
b = [[sp.Symbol(f"b{k}{j}") for j in range(dimensoes_camadas[k+1])]
     for k in range(profundidade)]

## listona que contem todos os w's e todos os b's
parametros = [*np.array(w).flatten(), *np.array(b).flatten()]

### Função de ativação

Agora vamos escolher a função de ativação que será usada na rede neural. Em um caso geral, nós podemos escolher uma função para cada par de neurônios. Mas nesse caso, por simplicidade, nós vamos escolher uma única função de ativação para todas as conexões da nossa rede neural. Mais especificamente, vamos escolher a função *sigmóide*, que é a função $\sigma: \mathbb R \to \mathbb R$ definida por $\sigma(x) = \frac{1}{1+e^{-x}}$. Também é possível definir outras funções (como *ReLU*, *tanh*, etc.) nessa célula.


In [ ]:
# Definir a função de ativação:

x = sp.Symbol('x')
sigma = 1/(1 + sp.exp(-x))

### Rede neural

Agora vamos definir a rede neural *per se*.  Por definição, para cada camada $k \in \{0, 1, \dotsc, n-1\}$, a rede multiplica o vetor que sai da camada anterior pela matriz de pesos $w_k$, soma o resultado com o vetor de vieses $b_k$, e aplica a função de ativação $\sigma$ a cada coordenada do vetor resultante.


In [ ]:
# Função que avalia a rede neural em um vetor

def rede_neural (entrada):
    vetor = entrada
    for k in range(profundidade):
        vetor = [sum([w[k][i][j] * vetor[j] for j in range(dimensoes_camadas[k])]) + b[k][i]
                 for i in range(dimensoes_camadas[k+1])]
        vetor = [sigma.subs(x, vetor[i]) for i in range(dimensoes_camadas[k+1])]
    return vetor

## Dados

Lembre que uma rede neural pode ser ajustada a um conjunto de dados. Nessa parte, vamos definir um tal conjunto, formatá-lo adequadamente e realizar verificações para garantir sua consistência com a arquitetura da rede neural definida acima.


Para começar, considere um conjunto de pares da forma
$$
    D = \{(x_1, y_1), (x_2, y_2), \dotsc, (x_m, y_m)\},
$$
onde $m > 0$ é a quantidade de *samples*, $x_1, x_2, \dotsc, x_m \in \mathbb{R}^{d_0}$ são vetores-entrada, e $y_1, y_2, \dotsc, y_m \in \mathbb{R}^{d_n}$ são vetores-saída. O objetivo do ajuste (ou treinamento) da rede neural é encontrar parâmetros (pesos e vieses) que façam com que a imagem do vetor $x_i$ pela rede neural seja o mais próximo possível do vetor $y_i$, para todo $i \in \{1, 2, \dotsc, m\}$.


In [ ]:
# Definir os conjunto de dados aos quais a rede será ajustada:

dados = {(0, 0), (1, 1), (2, 0), (3, 1)}

Antes de continuar, vamos tratar e validar o conjunto de dados acima. Para começar, vamos verificar que cada um dos dados é formado por duas partes, uma que representa um vetor-entrada e a outra que representa um vetor-saída.


In [ ]:
# Verificação de que todos os dados têm duas partes

for dado in dados:
    if len(dado) != 2:
        raise ValueError(f"Erro nos dados: o ponto {dado} tem {len(dado)} partes, mas deveria ter 2 partes.")

Se alguma mensagem tiver sido impressa na saída da célula acima, nós não poderemos continuar. Caso contrário, vamos formatar os dados como pares de tuplas, uma que representa um vetor-entrada e a outra que representa um vetor-saída.


In [ ]:
# Formatação do conjunto de dados

dados_formatados = []

for dado in dados:

    dado_entrada = dado[0]
    if isinstance(dado_entrada, (int, float)):
        dado_entrada = (dado_entrada,)
    elif isinstance(dado_entrada, (tuple, list)):
        dado_entrada = tuple(dado_entrada)
    else:
        raise ValueError(f"Erro na formatação dos dados: {dado} é de tipo {type(dado)}.")

    dado_saida = dado[1]
    if isinstance(dado_saida, (int, float)):
        dado_saida = (dado_saida,)
    elif isinstance(dado_saida, (tuple, list)):
        dado_saida = tuple(dado_saida)
    else:
        raise ValueError(f"Erro na formatação dos dados: {dado} é de tipo {type(dado)}.")

    dado = (dado_entrada, dado_saida)
    dados_formatados.append(dado)

dados = dados_formatados

Se alguma mensagem tiver sido impressa na saída da célula acima, nós não poderemos continuar. Caso contrário, vamos verificar se as dimensões dos vetores-entrada de todos os dados estão coincidindo com a dimensão da camadas de entrada da rede, e analogamente, se as dimensões dos vetores-saída de todos os dados estão coincidindo com a dimensão da camada de saída da rede.


In [ ]:
# Verificação da consistência das dimensões

for dado in dados:
    if len(dado[0]) != dimensoes_camadas[0] or len(dado[1]) != dimensoes_camadas[-1]:  # Corrigido
        raise ValueError(f"A dimensão do dado {dado} não está consistente com a arquitetura da rede.")

## Ajuste da rede neural

Lembre que uma das características mais importantes das redes neurais para aplicações é que elas são suficientemente flexíveis para se ajustar bem a diversos conjuntos de dados. Como consequência, isso permite que elas sejam úteis para modelar diversos fenômenos. Nessa parte, vamos calcular os parâmetros da rede que fazem com que ela se ajuste melhor ao conjunto de dados fornecido acima. Para isso, nós definiremos uma função erro e utilizaremos o Método de Newton para encontrar uma aproximação para os valores dos parâmetros que minimizam esse erro.


### Função erro (*loss function*)

A função erro é obtida comparando as imagens da rede neural nos vetores-entrada do conjunto de dados com os seus respectivos vetores-saída. Essa função pode ser definida de diversas maneiras. Nós vamos defini-la como a soma dos quadrados das diferenças entre esses dois vetores.


In [ ]:
# Função que calcula o erro entre a imagem da rede e os dados de entrada

def erro (dados):
    saida = [rede_neural(dado[0]) for dado in dados]
    dados_saida = [dado[1] for dado in dados]
    L = sum([(saida[i][j] - dados_saida[i][j])**2 for i in range(len(dados)) for j in range(dimensoes_camadas[-1])])
    return L

### Minimização da função erro

Lembre que ajustar uma rede neural consiste em encontrar os parâmetros dessa rede que minimizam a função erro $L$. Como essa função depende de várias variáveis (os parâmetros da rede), lembre da disciplina de *Cálculo em Várias Variáveis* que os seus pontos de mínimo anulam seu gradiente. Depois observe que a equação $\nabla L = (0, \dotsc, 0)$ é equivalente a um sistema de equações que não é necessariamente linear. Um dos métodos numéricos mais usados na resolução desse tipo de sistema é o chamado *Método de Newton*. Esse é um método iterativo que constrói uma sequência de aproximações para uma solução do sistema de equações desejado.
<!--Para isso, nós começamos com uma aproximação inicial (que, nesse caso, nós vamos escolher como $Z_0 = (0, \dotsc, 0)$) e definimos a próxima aproximação como a solução do sistema linear
$$
    J(Z_n) X = J(Z_n) Z_n - \nabla L(Z_n),
$$
onde $J(Z_n)$ é a matriz Jacobiana de $\nabla L$ (ou, equivalentemente, a matriz Hessiana de $L$) avaliada na $n$-ésima aproximação $Z_n$.-->


Antes de começarmos a aplicar o Método de Newton à rede neural construida acima, vamos estabelecer alguns *critérios de parada* para ele; ou seja, vamos estabelecer quando nós consideraremos que uma aproximação para a solução da equação $\nabla L = (0, \dotsc, 0)$ será boa. Existem vários critérios de parada possíveis. Nós escolheremos o seguinte.

Dados $\delta > 0$, $\varepsilon > 0$ e $M > 0$, nós consideraremos que uma aproximação para a solução da equação $\nabla L = (0, \dotsc, 0)$ é boa quando:

 * as aproximações não estiverem mudando muito, ou seja, $\|Z_{i+1} - Z_i\| < \delta$,

 * ou o gradiente estiver próximo de zero, ou seja, $\| \nabla L(Z_i)\| < \varepsilon$,

 * ou o número de iterações já for muito grande, ou seja, $i > M$.

Agora vamos escolher valores concretos para $\delta$, $\varepsilon$ e $M$.


In [ ]:
# Parâmetros dos critérios de parada:

delta = 10**(-5)
epsilon = 10**(-5)
max_iteracoes = 100

Para concluir, vamos implementar o Método de Newton.


In [ ]:
# Cálculo da função erro
L = erro(dados)

# Gradiente da função erro
gradL = [sp.diff(L, p) for p in parametros]

# Aproximação inicial
z = np.zeros(len(parametros), dtype=np.float32)

# Lista que vai guardar as aproximações do Método de Newton
Z = []
Z.append(z)

In [ ]:
# Função que calcula a matriz Jacobiana

def jacobiana (funcao, vetor):
    n = len(parametros)

    if len(funcao) != n:
        raise ValueError(f"A função dada tem {len(funcao)} entradas, mas deveria ter {n} entradas.")

    elif len(vetor) != n:
        raise ValueError(f"O vetor dado tem {len(vetor)} entradas, mas deveria ter {n} entradas.")

    else:
        matriz = np.zeros([n, n])

        for i in range(n):
            for j in range(n):
                fi = funcao[i]
                xj = parametros[j]
                dfidxj = sp.diff(fi, xj)
                matriz[i][j] = sp.lambdify(parametros, dfidxj)(*vetor)

        return matriz

In [ ]:
# Função que calcula o lado direito da equação J(Z) X = J(Z)Z - ∇L(Z)

def lado_direito (funcao, vetor):
    if len(vetor) != len(parametros):
        raise ValueError(f"O vetor {vetor} tem {len(vetor)} entradas, mas deveria ter {len(parametros)} entradas.")
    else:
        DFZZ = jacobiana(funcao, vetor) @ vetor
        F = sp.lambdify(parametros, funcao)
        FZ = F(*vetor)
        saida = DFZZ - FZ
        return np.array(saida, dtype=np.float32)

In [ ]:
# Loop completo do Método de Newton

F = sp.lambdify(parametros, gradL)

A = jacobiana(gradL, Z[-1])
B = lado_direito(gradL, Z[-1])
z = np.linalg.solve(A, B)
Z.append(z)

while (np.linalg.norm(Z[-1] - Z[-2], ord=1) > delta and
       np.linalg.norm(F(*Z[-1]), ord=1) > epsilon and
       len(Z) < max_iteracoes):

    A = jacobiana(gradL, Z[-1])
    B = lado_direito(gradL, Z[-1])
    z = np.linalg.solve(A, B)
    Z.append(z)

for i in range(len(Z)):
    print(f"Z_{i} = {Z[i]}")